In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d shivanandmn/multilabel-classification-dataset

 87% 10.0M/11.4M [00:01<00:00, 12.0MB/s]
100% 11.4M/11.4M [00:01<00:00, 7.52MB/s]


In [ ]:
!unzip /content/multilabel-classification-dataset.zip

Archive:  /content/multilabel-classification-dataset.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
train_path = '/content/train.csv'
test_path = '/content/test.csv'

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import re

In [ ]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.strip(' ')
    return text

In [ ]:
train_df['ABSTRACT'] = train_df['ABSTRACT'].map(lambda com : clean_text(com))

In [ ]:
X = train_df.ABSTRACT
test_X = test_df.ABSTRACT

In [ ]:
# import and instantiate TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
vect = TfidfVectorizer(max_features=5000,stop_words='english')
vect

TfidfVectorizer(max_features=5000, stop_words='english')

In [ ]:
X_vec = vect.fit_transform(X)
X_vec

<20972x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 1182453 stored elements in Compressed Sparse Row format>

In [ ]:
test_X_vec = vect.transform(test_X)

In [ ]:
X_vec

<20972x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 1182453 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

submission_chains = pd.read_csv('/content/sample_submission.csv')

# create a function to add features
def add_feature(X, feature_to_add):
    '''
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    '''
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

In [ ]:
cols_target = ['Computer Science','Physics','Mathematics','Statistics','Quantitative Biology','Quantitative Finance']


In [ ]:
import tensorflow as tf
from sklearn.metrics import accuracy_score



for label in cols_target:
    print('... Processing {}'.format(label))
    y = train_df[label].values.astype(float)


    X_vec_dense = X_vec.toarray()
    test_X_vec_dense = test_X_vec.toarray()


    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(X_vec_dense.shape[1],)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])


    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


    model.fit(X_vec_dense, y, epochs=20, batch_size=32)


    y_pred_X = (model.predict(X_vec_dense) > 0.5).astype(float)
    print('Training Accuracy is {}'.format(accuracy_score(y, y_pred_X)))


    test_y_prob = model.predict(test_X_vec_dense)
    submission_chains[label] = test_y_prob


    X_dtm = add_feature(X_vec_dense, y)
    print('Shape of X_dtm is now {}'.format(X_dtm.shape))


... Processing Computer Science
Epoch 1/20
656/656 [==============================] - 3s 4ms/step - loss: 0.6052 - accuracy: 0.7321
Epoch 2/20
656/656 [==============================] - 2s 3ms/step - loss: 0.4956 - accuracy: 0.8276
Epoch 3/20
656/656 [==============================] - 2s 3ms/step - loss: 0.4362 - accuracy: 0.8465
Epoch 4/20
656/656 [==============================] - 2s 3ms/step - loss: 0.3998 - accuracy: 0.8549
Epoch 5/20
656/656 [==============================] - 2s 3ms/step - loss: 0.3752 - accuracy: 0.8609
Epoch 6/20
656/656 [==============================] - 2s 3ms/step - loss: 0.3573 - accuracy: 0.8657
Epoch 7/20
656/656 [==============================] - 3s 4ms/step - loss: 0.3435 - accuracy: 0.8696
Epoch 8/20
656/656 [==============================] - 2s 3ms/step - loss: 0.3325 - accuracy: 0.8737
Epoch 9/20
656/656 [==============================] - 2s 3ms/step - loss: 0.3235 - accuracy: 0.8762
Epoch 10/20
656/656 [==============================] - 2s 3ms/step -

In [ ]:
"""
Epoch 20/20
656/656 [==============================] - 2s 3ms/step - loss: 0.0191 - accuracy: 0.9924
656/656 [==============================] - 2s 2ms/step
Training Accuracy is 0.9927522410833493
""